In [ ]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import uncertainties as unc
import table_utils as tu

import pint.derived_quantities as dq
#See num2tex (https://github.com/AndrewChap/num2tex)
# > pip install num2tex
from num2tex import num2tex

pint.logging.setup(level="ERROR")

In [ ]:
EFAC_dict = {
    "J0405+3347": 1.0597,
    "J0742+4110": 1.3308,
    "J1018-1523": 1.1679,
    "J1045-0436": 1.3027,
    "J1122-3546": 1.2736,
    "J1221-0633": 1.2066,
    "J1317-0157": 1.2961,
    "J1742-0203": 1.0817,
    "J2017-2737": 1.6155,
    "J2018-0414": 1.0262,
    "J2022+2534": 1.0354,
    "J2039-3616": 1.0349,
}

In [ ]:
# Auto-populate rotational/timing params table
for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    f0 = tu.uf(mo['F0'])
    f1 = tu.uf(mo['F1'])
    epoch = int(mo['PEPOCH'].value)
    span = f"{int(mo['START'].value)}--{int(mo['FINISH'].value)}"
    rms_us = f"{fo.resids.rms_weighted().to(u.us).value:.1f}"
    ntoa = mo['NTOA'].value
    efac = EFAC_dict[psr]
    # '\' is a special python character, but prefixing with '\' turns it into a normal character print('\\') -> '\'
    out_str = f"{psr_tex} & {f0} & {f1} & {epoch} & {span} & {rms_us} & {ntoa} & {efac} \\\\"
    print(out_str.replace(" \\times ","\\times"))

In [ ]:
# Auto-populate derived common properties table
for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    # Calculate derived quantities with PINT
    f0,f0err,f1,f1err = (mo['F0'].quantity,mo['F0'].uncertainty,mo['F1'].quantity,mo['F1'].uncertainty)
    p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
    p_tex = tu.ufve(p.value,perr.value)
    pd_tex = tu.ufve(pd.value,pderr.value)
    age = f"{num2tex(dq.pulsar_age(f0,f1).value):.1e}" # yr
    bsurf = f"{num2tex(dq.pulsar_B(f0,f1).value):.1e}" # G
    edot = f"{num2tex(dq.pulsar_edot(f0,f1).value):.1e}" # erg/s
    gcoord = mo.coords_as_GAL()
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,mo['DM'])
    
    out_str = f"{psr_tex} & {p_tex} & {pd_tex} & {age} & {bsurf} & {edot} & {dmdist_ne:.1f} & {dmdist_ymw:.1f} \\\\"
    
    print(out_str.replace(" \\times ","\\times"))

In [ ]:
# Auto-populate coordinates/DM table
for k in EFAC_dict.keys():
    par_path = f"data/{k}_swiggum+22.par"
    tim_path = f"data/{k}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    # J1742 doesn't have quoted DM error (estimate one?)
    if k == 'J1742-0203':
        dm = f"{mo['DM'].value:.2f}"
    else:
        dm = tu.uf(mo['DM'])
        
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    elat = tu.uf(mo['ELAT']).replace('-','$-$')
    eqcoord = mo.coords_as_ICRS()
    ra = tu.format_ra(eqcoord)
    dec = tu.format_dec(eqcoord)
    gcoord = mo.coords_as_GAL()
    gl = f"{gcoord.l.deg:.2f}"
    gb = f"{gcoord.b.deg:.2f}".replace('-','$-$')
    print(f"{psr_tex} & {tu.uf(mo['ELONG'])} & {elat} & {dm} & {ra} & {dec} & {gl} & {gb} \\\\")

In [ ]:
psr_names = ['J0742+4110','J2022+2534','J2039-3616']

for pn in psr_names:
    par_path = f"data/{pn}_swiggum+22.par"
    tim_path = f"data/{pn}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    pmelong = f"{tu.uf(mo['PMELONG'])}".replace("-","$-$")
    pmelat = f"{tu.uf(mo['PMELAT'])}".replace("-","$-$")

    gcoord = mo.coords_as_GAL()
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,mo['DM'])

    for ii,dmdist in enumerate([dmdist_ne,dmdist_ymw]):
        dd,de = (dmdist,dmdist*0.3)*u.kpc
        dist = tu.ufve(dd.value,de.value)
        pmtot,pmtoterr = tu.PMtot_err(mo['PMELONG'],mo['PMELAT'])
        vt,vterr = tu.Vtrans_err(pmtot,pmtoterr,dd,de)
        vt_str = tu.ufve(vt.value,vterr.value)

        f0,f0err,f1,f1err = (mo['F0'].quantity,mo['F0'].uncertainty,mo['F1'].quantity,mo['F1'].uncertainty)
        p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
        pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
        pds = f"{pdshk*1e21:.2f}"
        pdgal = tu.pd_gal(p,gcoord,dd).decompose()
        pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

        # Am I doing the math here correctly?
        pdint = pd-(pdshk+pdgal)
        pdi = f"{pdint.value*1e21:.2f}"

        # Re-derive age, bsurf, edot with intrinsic P-dot
        f1 = -1.0*pdint/p**2
        age = f"{dq.pulsar_age(f0,f1).value*1e-9:.1f}" # Gyr
        bsurf = f"{dq.pulsar_B(f0,f1).value*1e-8:.1f}" # 10^8 G
        edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.1f}" # 10^33 erg/s

        if ii == 0:
            print(f"{psr_tex} & {pmelong} & {pmelat} & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")
        elif ii == 1:
            print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\")

In [ ]:
# Example thing
par_path = "data/J1018-1523_swiggum+22.par"
tim_path = "data/J1018-1523_swiggum+22.tim"
mo = model.get_model(par_path)
to = toa.get_TOAs(tim_path,model=mo)
fo = pint.fitter.WLSFitter(to,mo)
x = fo.fit_toas()

In [ ]:
binary_psrs = ['J0742+4110','J1045-0436','J1221-0633','J1317-0157','J2022+2534','J2039-3616']

for bp in binary_psrs:
    par_path = f"data/{bp}_swiggum+22.par"
    tim_path = f"data/{bp}_swiggum+22.tim"
    mo = model.get_model(par_path)
    to = toa.get_TOAs(tim_path,model=mo)
    fo = pint.fitter.WLSFitter(to,mo)
    x = fo.fit_toas()
    
    psr = mo['PSR'].value
    psr_tex = psr.replace('-','$-$')
    a1_tex = tu.uf(mo['A1'])
    pb_tex = tu.uf(mo['PB'])
    tasc_tex = tu.uf(mo['TASC'])
    eps1_tex = tu.uf(mo['EPS1'])
    eps2_tex = tu.uf(mo['EPS2'])
    
    # Derive mass function and Mc,min
    fm = dq.mass_funct(mo['PB'].quantity, mo['A1'].quantity)
    fm_tex = f"{num2tex(fm.value):.4e}" # Msun
    mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_tex = f"{mc.value:.2f}" # Msun

    outstr = f"{psr_tex} & {pb_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {fm_tex} & {mc_tex}"
    print(outstr.replace(" \\times ","\\times").replace(" -"," $-$"))